# Evaluations 101

（評価入門：Evals 101）

このレッスン（および続く8つの追加レッスン）では、AIアプリケーション開発における重要要素である「LLMユースケース評価（evaluations）」の書き方・考え方に焦点を当てます。

急速に進化するAIの世界では、大規模言語モデル（LLM）は多くの業界で強力なツールになりました。しかし、その潜在力を最大限に引き出すには、実装するだけでは不十分で、**慎重な評価と最適化** が必要です。そこで登場するのが、プロンプト評価（prompt evaluations）、通称 **「evals」** です。

以下は、評価の重要性を説明する社内 Solutions Architect チームの実際のコメント（引用）です：

>チームがモデル性能を測定できる（できない）ことが、LLMの本番ユースケースにおける最大の障害であり、プロンプト作りを科学ではなく“芸術”にしてしまっている

>evals は時間がかかるが、最初にやっておけば長期的には開発者の時間を節約でき、より良い製品をより早く世に出せる

開発者が評価を書かない主な理由は2つあります：
1. そもそも「評価」という発想に馴染みがない
2. 実際にどう実装すればよいかが分からない

このコースでは、評価とは何か、そしてどう書くか、その両方を説明します。

この導入レッスンでは前者（評価の概念）を扱い、評価とは何か、そして評価の書き方のアプローチを高レベルに概観します。

---


## Benchmarks
顧客向け評価（customer evaluations）の話に入る前に、多くの人が馴染みのある評価形態である **モデルベンチマーク** について整理しておきましょう。

モデルベンチマークは、AIの世界における標準テストのようなものです。たとえばSATスコアが大学に学生の学力の概観を与えることを意図しているのと同様に、モデルベンチマークは、AIモデルがさまざまなタスクに対してどの程度うまく動くかを幅広く把握するための指標です。

大規模言語モデルを作る企業は、モデルの能力を示すためにこうしたベンチマークを実施します。ARC、MMLU、TruthfulQA のような少し変わった名前のテストで高得点が出た、という話を見たことがあるかもしれません。これらは、基本的な読解から高度な推論・専門知識まで幅広くカバーします。異なるモデルの比較や、AI能力の全体的な進歩を追うのに役立ちます。ベンチマークスコアを載せたモデルカードを見たことがある方も多いでしょう：

![benchmarks.png](../images/benchmarks.png)

ただし、ベンチマークは「自慢」や能力の概観には良い一方で、それがすべてではありません。たとえばIQスコアを知っても、その人が一般的に賢いかもしれない（し、そうでないかもしれない）ことは分かっても、あなたの特定の仕事に向いているかどうかまでは分からないのと似ています。

---

## Customer evaluations

ピカピカのスイスアーミーナイフを買ったところを想像してください。便利な道具が何十個も付いていますが、あなたが主に使いたいのはキャンプで缶詰を開けることだとします。爪やすりやワインのコルク抜きができても嬉しいですが、肝心なのは「缶詰をどれだけ上手に開けられるか」ですよね。LLMにおけるプロンプト評価も、これと同じ発想です。

LLMはテキストに対する“超高性能”スイスアーミーナイフのようなもので、詩作からソフトウェアのコーディングまで言語を使った様々なことができます。しかし、カスタマーサポートメールへの返信や商品説明文の生成など、**特定のタスク** にLLMを使うなら、その仕事に対してどれだけうまく動くかを知る必要があります。

そこで登場するのが、プロンプト評価（prompt evaluations）、略して **「evals」** です。プロンプト評価（customer evaluations とも呼ばれます）は、LLMがあなたの **特定ユースケース** でどれだけうまく機能するかを測定するための体系的テストです。LLMの一般的な能力と、ビジネスアプリ固有の要求をつなぐ重要な橋渡しになります。堅牢な evals を実装することで、モデル＋プロンプトの組み合わせが一般ベンチマークを満たすだけでなく、必要なタスクで優れた性能を発揮することを確認できます。

評価の主なメリット：

* **反復的なプロンプト改善**：v1よりv2のプロンプトの方が、こちらのタスクで本当に良くなっているか？
* **デプロイ前後の品質保証**：最新のプロンプト更新で性能退行（regression）は起きていないか？
* **客観的なモデル比較**：Anthropic の最新モデルへ切り替えても、eval性能を維持（または改善）できるか？
* **コスト削減の可能性**：より安く速いモデルへ切り替えても、eval性能を維持できるか？

プロンプトを書いて最適化するとき、私たちは評価を軸にした反復プロセスを回します：

![process.png](../images/process.png)



* まずテストケースを作ります（詳しくはこのあと学びます）。
* 次に、特定ユースケース向けのプロンプトをざっくり書きます。
* そして、テストケースでプロンプトを試し、モデルがそのタスクをどれだけうまくできたか測定します（まずはベースラインとなるスコアが欲しい）。
* ベースラインができたら、プロンプトを変更して同じ手順を繰り返します。

evals の本質は、**プロンプト＋モデルの組み合わせの品質を、指標（メトリクス）で定量化すること** です。定量的な測定がなければ、プロンプトの変更が改善につながっているのかどうか判断できません。

---

## What goes into an eval?

よく設計されたプロンプト評価は、主に次の4要素で構成されます：

* **Example Input（入力例）**：モデルに与える指示や質問。実運用でアプリが遭遇する入力を正確に代表するよう設計することが重要です。
* **Golden Answer（正解／理想解）**：モデル出力の基準となる正しい（または理想的な）回答。高品質なゴールデンアンサーを作るには、正確さや妥当性のために、ドメイン専門家の協力が必要になることが多いです。
* **Model Output（モデル出力）**：入力プロンプトに基づいてLLMが実際に生成した応答。これをゴールデンアンサーと照合して評価します。
* **Score（スコア）**：その入力に対するモデル性能を表す定量／定性の値。採点方法はタスクの性質や選ぶ評価アプローチによって変わります。

![eval_diagram.png](../images/eval_diagram.png)

**一般には、良い結果を得るために少なくとも100組以上の「テストケース／ゴールデンアンサー」ペアを推奨します。ただし、このコースでは学習者のAPIコストを抑えるため、多くのevalはそれより少ないペア数で進めます。**

### Example eval data set

たとえば、LLMで顧客からの苦情を分類したいとします。ごく小さな評価データセットは次のようになります：

```py
eval_data = [
    {
        "complaint": "The app crashes every time I try to upload a photo",
        "golden_answer": ["Software Bug"]
    },
    {
        "complaint": "My printer isn't recognized by my computer",
        "golden_answer": ["Hardware Malfunction"]
    },
    {
        "complaint": "I can't figure out how to change my password",
        "golden_answer": ["User Error"]
    }
]
```
各入力の苦情に対して、対応するゴールデンアンサー（分類ラベル）を用意します。この例は後のレッスンで拡張し、評価の実行方法と採点方法を学びます。

---

## Grading approaches

適切な採点方法を選ぶことは、評価の有効性に直結します。各手法には強みがあり、向いているタスクの種類も異なります。

### Human-based grading

繊細な理解や主観的判断が必要なタスクでは、人手採点（human-based grading）が依然としてゴールドスタンダードです。この方法では、（多くの場合はドメイン専門家などの）人がモデル出力をレビューし、品質を評価してスコアを付けます。

人手採点は、トーン、創造性、複雑な推論、専門領域での事実性などの評価に強いです。自由記述型のタスクや、正誤が微妙な文脈に依存する場合に特に有用です。一方で、時間がかかり、特に大規模評価ではコストも高くなります。また、評価者間でばらつきが出る問題もあります。

人手採点の例：

* **Expert Review（専門家レビュー）**：専門家が応答の正確さ・深さを評価します。たとえば住宅ローンを扱う銀行チャットボットなら、弁護士が公正貸付法への準拠や条件説明の正確さを確認するかもしれません。皮膚科医が皮膚がんスクリーニング助言を評価し、識別の妥当性、緊急度の適切さ、最新研究との整合性を確認する、といった例もあります。
* **User Experience Panel（UXパネル）**：複数人が明確さ、役立ち度、エンゲージメントなど、人間の判断が必要な観点で評価します。

### Code-based grading

コード採点（code-based grading）は、プログラムでモデル出力を評価する方法です。明確で客観的な基準があるタスクに向いています。たとえば、テキストから特定のデータ項目を抽出するタスクなら、抽出結果が期待値と一致するかをコードでチェックできます。

コード採点の主な利点は、速度とスケーラビリティです。一度仕組みを作れば、何千件もの評価を高速かつ一貫して処理できます。ただし、繊細さや主観が絡む応答の評価には向きません。一般的な手法として、完全一致、キーワードの有無、正規表現によるパターンマッチなどがあります。

コード採点の例：
* **完全一致（Exact String Match）**：最も厳格な形式で、モデル出力がゴールデンアンサーと1文字単位で完全に一致する必要があります。地理クイズで「フランスの首都は？」に対し、受理される答えが「Paris」だけ、のようなイメージです。
* **キーワード含有（Keyword Presence）**：順序や文脈に依存せず、重要な単語・フレーズが含まれるかをチェックします。たとえばスマートホームのサポートチャットで「温度調節器をリセットするには？」に対し、「hold」「button」「5 seconds」「blinking light」が含まれることを要件にする、などです。
* **正規表現（Regex）**：より複雑なパターンをチェックできます。たとえばクレジットカード審査の説明で、スコアと結果が両方含まれるように `"Your credit score of \\d{3} (qualifies|does not qualify) you for our \\w+ Card"` のようなパターンを要求できます。
* その他いろいろ！

### LLM-based grading

LLM採点（LLM-based grading）は、コード採点と人手採点の中間に位置づけられる方法です。別のLLM（または同じLLM）に、出力の評価をさせます。丁寧に採点用プロンプトを作ることで、LLMの言語理解を活かして幅広い基準を評価できます。

この方法は、コード採点より複雑で主観的な評価を扱え、人手採点より速くスケールしやすい一方で、信頼できる結果を得るにはプロンプト設計のスキルが必要です。また採点側LLMがバイアスを持ち込むリスクもあります。

モデル採点の例：
* **要約品質（Summarization Quality）**：この要約はどれだけ簡潔で正確か？
* **トーン評価（Tone Assessment）**：ブランドガイドラインやトーンに沿っているか？
* **その他あらゆる品質**：独自ルーブリックを定義し、「どれだけ謝罪しているか」「競合に言及していないか」など、想像できる任意の指標で採点させられます。


---

## Conclusion

堅牢なプロンプト評価を実装することは、効果的なLLMアプリケーションを作る上で重要なステップです。テストと改善を体系的に回すことで、アプリが求める要件に対して一貫して高品質な結果を出せることを担保できます。良い evals を作るために投資した時間は、性能向上・最適化の容易さ・LLM活用に対する自信として回収されます。

次は、evals の具体的な書き方を学びましょう！